# Walmart API overview
##### Search API used for look ups. We will use this first
##### Taxonomy API used for classifications.  We want to find a way to count classifications

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import urllib
import json
import sys
import pprint

# We are using 2 api Keys for redundancy and because
# there is a limit to the number of calls
api_key = "4tsbjxvbrwnpcjdsh277csqu"
api_alt = "anrtcd5dheh9758z5wgdy677"

# Base url for all API look ups
base_url = "http://api.walmartlabs.com/v1/" 

# Variables to specify the API
search = "search?"
paginated_products = "paginated/items?"

# Base url for search
search_url = base_url + search
paginated_url = base_url + paginated_products



# Search API

In [ ]:
# Load search criteria for Walmart Search API.
# reference URL: https://developer.walmartlabs.com/docs/read/Search_API

format = "json"
categoryId = "3944"
query = "laptop"
numItems = 25 # max 25. 10 is default
start = 1
sort = "price"
order = "desc"
facet = "on" # boolean
min = str(600) # lower end of range for search within price range
max = str(700) # higher end of range price for search with price range
facet_range = (f"price:[{min}%20TO%20{max}]") # note the format [num1:num2] does not work. Use %20TO%20 for ":"



# NOTE: we are using 2 methods to create urls. Because the params argument does not format the output
# in the manner needed to get the API to work.  This is used only for price range lookups

# manual url. Used for price range lookup only
manual_url = (f"{search_url}apiKey={api_key}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")

print(manual_url)

# API parameters for other lookups
params = {
    "apiKey": api_key,
    #"start": start, # note keep turned off.  We can't get this to work yet
    "format": "json", # json|xml
    "categoryId": categoryId,
    "query": query,
    "numItems": numItems,
    #"sort": sort, #allowed sort types are [relevance, price, title, bestseller, customerRating, new]
    #"order": order,  #allowed values are [asc, desc]
    #"facet":facet, # note keep turned off.  We can't get this to work yet
    #"facet.range":facet_range    # note keep turned off.  We can't get this to work yet
}

In [6]:
# Loop by pricing parameters to pull out entire data set for a product category
# to save time we are going to hard code some numbers then go back later if we have time

# Number of items found in our search
# there are 20,747 total results for laptop in electronics
# step 0 there are 3089 results in the price range $0-$200
# step 1 there are 3015 results in the price range $201-$400
# step 2 there are 2053 results in the price range $401-$600
# step 3 there are 2995 results in the price range $601-$1000
# step 4 there are 2021 results in the price range $1001-$1400
# step 5 there are 2726 results in the price range $1401-$1800
# step 6 there are 1873 results in the price range $1801-$2200
# step 7 there are 1011 results in the price range $2201-$2600
# step 8 there are 863 results in the price range $2601-3000
# step 9 there are 1096 results in the price range above $3000

category_total = 20747 #hard coded from broad search
step0 = 200*25//3089
step1 = 200*25//3015
step2 = 200*25//2053
step3 = 400*25//2995
step4 = 400*25//2021
step5 = 400*25//2726
step6 = 400*25//1873
step7 = 400*25//1011
step8 = 400*25//863

# create a function to perform the price search

def price_data(lowRange, highRange):
    facet_range = (f"price:[{lowRange}%20TO%20{highRange}]")
    price_url = (f"{search_url}apiKey={api_alt}&format={format}&categoryId={categoryId}&query={query}&numItems={numItems}&facet={facet}&facet.range={facet_range}")
    walmart_data2 = requests.get(price_url)
    walmart_data2 = walmart_data2.json()
    print(price_url)
    try:       
        a = []
        if not os.path.isfile('Walmart_datasets2/walmart_more.json'):
            a.append(walmart_data2)
            with open('Walmart_datasets2/walmart_more.json', mode='w') as f:
                f.write(json.dumps(walmart_data2, indent=4))
        else:
            with open('Walmart_datasets2/walmart_more.json') as feedsjson:
                feeds = json.load(feedsjson)

            feeds.append(walmart_data2)
            with open('Walmart_datasets/walmart_more.json', mode='w') as f:
                f.write(json.dumps(feeds, indent=4))

    except:
        print ("Skipped line")
                


In [11]:
# Use for loop to go through prices in the steps defined

searchCount = 0
itemCount = 0
low = 728
high = 728

for itemCount in range (1,12000):
    itemCount = (searchCount+1)*25
    if high < 200:
        high = low + step0
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 400:
        high = low + step1
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 600:
        high = low + step2
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 1000:
        high = low + 25
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")            
    elif high < 1400:
        high = low + step4
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")    
    elif high < 1800:
        high = low + step5
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 2200:
        high = low + step6
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")        
    elif high < 2600:
        high = low + step7
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 3000:
        high = low + step8
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    elif high < 8000:
        high = low + step9
        price_data(low,high)
        low = high
        searchCount += 1
        print(f"search / item count = {searchCount} / {itemCount}")
    else:
        print ('finished')
        print (f'item count = {itemCount}')
        print (f'final search count = {searchCount}')
        print (f'final high price is ${high}')
        break
      



http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[728%20TO%20753]
Skipped line
search / item count = 1 / 25
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[753%20TO%20778]
Skipped line
search / item count = 2 / 50
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[778%20TO%20803]
Skipped line
search / item count = 3 / 75
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[803%20TO%20828]
Skipped line
search / item count = 4 / 100
http://api.walmartlabs.com/v1/search?apiKey=anrtcd5dheh9758z5wgdy677&format=json&categoryId=3944&query=laptop&numItems=25&facet=on&facet.range=price:[828%20TO%20853]
S

KeyboardInterrupt: 

In [10]:
from pprint import pprint
import os
filepath = os.path.join('Walmart_datasets2/walmart_more.json')
with open(filepath) as jsonfile:
    json_data = json.load(jsonfile)
    list_length = (len(json_data['items']))
    print (f"Count of items = {list_length}")
    for i in range(list_length):
        try:
            print (json_data['items'][i][salePrice])
            print (f"count = {i}")
        except:
            print(f"skipped entry {i}")
    


Count of items = 25
skipped entry 0
skipped entry 1
skipped entry 2
skipped entry 3
skipped entry 4
skipped entry 5
skipped entry 6
skipped entry 7
skipped entry 8
skipped entry 9
skipped entry 10
skipped entry 11
skipped entry 12
skipped entry 13
skipped entry 14
skipped entry 15
skipped entry 16
skipped entry 17
skipped entry 18
skipped entry 19
skipped entry 20
skipped entry 21
skipped entry 22
skipped entry 23
skipped entry 24
